In [2]:
import requests
import time
import os
import sys
import re
import io
import json
import csv
import openpyxl
import pandas as pd
import numpy as np
import kss
import ast
from pykospacing import spacing
from konlpy.tag import Komoran

In [4]:
#각 키워드에 해당하는 내용 정리
#예시: ['결혼', '남자', '남편', '본인', '부모', '시간', '엄마', '여자', '자기', '친구']
from IPython.core.display import display, HTML
from konlpy.tag import Okt

#키워드에 해당하는 제목, 본문, 댓글 출력, 추후 iteration 돌며 10가지 모두 할 것
key = '자기'

final = pd.DataFrame(columns = ['contents'])

f = pd.read_csv('natepann.csv')

#키워드 본문, 제목, 댓글 돌며 확인
is_key = f['본문'].str.contains(key, na=False) | f['제목'].str.contains(key, na=False) | f['댓글'].str.contains(key, na=False)


f_key = f[is_key]
f_key = f_key.reset_index()
f_key

,index,제목,본문,댓글
0,0,너네엄 마라고 하는 시 어머니,어느 순간부터 시 어머니가 친 정 엄마를 지칭해서 말 할 때 너네엄 마라고 하더라고...,으이그 진짜 무식 한 여편 네 어른한테 이런 표현 하기 싫지만 대우해 주고 싶지 않...
1,1,임신에 대한 친정과 시댁의 온도차,추가 어젯밤에 서운한 맘에 글 써놓고 오늘은 남편 하고 남편 친구하고 도련님 셋이 ...,재미도 없고 마지막 줄 진짜 인성 나오네 주 작 오지네 임신 초기에 회 절대 못 먹...
2,2,예비 신랑 탈모 때문에 너무 고민입니다,안녕하세 요저는 예랑이와 년 정도 연애 하고 몇 번의 헤어짐을 겪고 결혼 약속을 하...,하루 빨리 약 복용하시길 추천 드립니다 약 먹는 순간부터 현 머리 상태가 유지된다고...
3,4,한 회사에서 꾸준히 오래 다니는 법,안녕하세요 사회 초년생인데요일은 년 이상 오래 하지를 못해요 일을 못하는 게 아니고...,아무 생각 없이 입사하셨나 봐요 지금부터 내 커리어’를 생각하고 내가 무얼 하는 사...
4,5,회계사 시험 붙더니 헤어지자는 남자친구,근 년간을 남자친구 뒷바라지하고 많이 챙겨줬었는데 요 저희끼리 서로 결혼은 언제 할...,이런 사람도 있고 저런 넘도 있을 텐데 글쓴이에게는 저런 넘이 걸렸네요 사람도 봐가...
5,6,얘들아 남자들 생떼 쓰는 것 좀 막아보자 청원 부탁,내가 설명이 좀 부족했던 거 같음 요즘 재재 손 모양도 그렇고 바연길 테러도 그렇고...,왜 딜 박히니까 짜증나 니네 입지 좁히는 것 같아서 근데 이건 페미들한테 고마워 해...
6,7,엄마가 필리핀 사람이라 비혼주의가 됐어요,부모님을 창피해하고 싶지는 않지만 아버지는 농부시고 어머니는 필리핀 분이세요가난한 ...,아이는 죄가 없지 부모가 선택했을 뿐 아이가 선택한 건 무엇도 없으니까 근데 나는 ...
7,9,시댁 문제로 남편과 심각하게 싸웠어요,안녕하세요 제가 남편이랑 크게 부부싸움을 했는데 제 생각이 잘못된 건지 진짜 의견 ...,다른 건 모르겠구요 일단 부부 싸움에 부모님 소환은 안 좋은 신호 맞아요 약간 저희...
8,12,예비 시동생 과거 파혼 결혼,안녕하세요 년차 연애 후 결혼 진행 중에 있는 예신이에 요 주변에 말하면 결코 축복...,나도 같은 급이에요 라고 말하고 싶은 건가님이랑 님네 집안도 그렇게 수준 떨어지는 ...
9,14,년 계약한 세입자가 년 더 살겠다는데 가능한가요,안녕하세 요인생 처음 마련한 집 년 아파트 월세 주고 저희 가족이 입주할 예정이었는...,계약 시 부동산에서 년 계약이라도 임차인이 원하면 최소 년은 살게 된다고 알려주지 ...


In [ ]:
final_key = pd.DataFrame(columns = ['tokenized'])
komoran = Komoran()

for i in range(len(f_key)):
    try:
        st = f_key.loc[i, '제목']
        final_key.loc[i, 'tokenized'] = st
        ct = f_key.loc[i, '본문']
        final_key.loc[i, 'tokenized'] += ct
        
        cm = f_key.loc[i, '댓글']
        #str Concatenation 문제 발생시 주석처리한 부분처럼 str() 처리하면 해결 가능
        final_key.loc[i, 'tokenized'] += cm
        #final_key.loc[i, 'tokenized'] += str(cm)
        
    except KeyError:
        print("error")

keyword_content = pd.DataFrame(columns = ['content','positive', 'negative', 'neutral'])


for i in range(len(final_key)):
    try:
        fc = final_key.loc[i, 'tokenized']#.replace(" ", "")
        fc = spacing(fc)
        #fc = str(komoran.morphs(fc))
        
        if i == 0:
            keyword_content.loc[0,'content'] = fc
        else:
            keyword_content.loc[0,'content'] += fc
        
    except KeyError:
        print("error")

keyword_content

In [4]:
# 감성사전에서 감성분석
f = open('SentiWord_Dict.txt', 'r', -1, 'utf-8')
lines = f.readlines()

score_dict = []

for line in lines:
    line_splited = line.split()
    score = int(line_splited[-1])
    word = ''
    for frac in line_splited[:-1]:
        word = word + ' ' + frac
    
    word = word[1:]
    score_dict.append([word, score])

In [5]:
#keys = []

keyword_content['positive'] = 0
keyword_content['negative'] = 0
keyword_content['neutral'] = 0

#komoran = Komoran()
#keys = spacing(keyword_content.loc[0, 'content'])
#keys = = str(komoran.nouns(keys)

#keyword_content.loc[0, 'content'] = spacing(keyword_content.loc[0, 'content'])
keyword_content.loc[0, 'content'] = str(komoran.nouns(keyword_content.loc[0, 'content']))


stopwords = ['하다', '없다', '있다', '되다', '아니다', '같다', '이다', '않다', '그렇다', 
             '이렇다', '싶다', '다', '것', '그', '이', '거', '니다']


pos_list = ast.literal_eval(keyword_content.loc[0, 'content'])
    
final = []
    
for j in range(len(pos_list)):
    if pos_list[j] not in stopwords:
        final.append(pos_list[j])
    
keyword_content.loc[0, 'content'] = str(final)
#keys = str(final)

In [6]:
keyword_content.insert(0, "Keyword", key)

In [7]:
#keyword_content.head()

In [8]:
#tokens.append(keyword_content.loc[0, 'content'])
#tokens.extend(keys)

tokens = ast.literal_eval(keyword_content.loc[0, 'content'])

try:
    for token in tokens:
        for dict_word in score_dict:
            if dict_word[0] == token:
                if dict_word[1] > 0:
                    keyword_content.loc[0, 'positive'] += dict_word[1]
                elif dict_word[1] < 0:
                    keyword_content.loc[0, 'negative'] += dict_word[1]
                else:
                    keyword_content.loc[0, 'neutral'] += 1 #중립어는 개수 세기    
except KeyError:
    print("error")

In [9]:
keyword_content[keyword_content.columns.difference(['content'])]

,Keyword,negative,neutral,positive
0,자기,-427,6,225


In [6]:
#% 시각화
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import font_manager, rc
import platform

#한글 꺠짐 처리
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')
    
matplotlib.rcParams['axes.unicode_minus'] = False
pos = keyword_content.loc[0, 'positive']
neg = str(keyword_content.loc[0, 'negative']).replace("-", "")
neu = keyword_content.loc[0, 'neutral']

size = [pos, neg, neu]

label = ['Positive','Negative','Neutral']

plt.axis('equal')
plt.pie(x=size, labels=label, autopct='%.2f%%')

plt.title("키워드 : " + key)
plt.legend(loc="upper left")
plt.show()

NameError: name 'keyword_content' is not defined